# Linear Layer

In [1]:
import torch

## Raw Linear Layer

$$\begin{gathered}
y=x\cdot{W}+b, \\
\text{where }x\in\mathbb{R}^{N\times{n}}\text{, }y\in\mathbb{R}^{N\times{m}}. \\
\\
\text{Thus, }W\in\mathbb{R}^{n\times{m}}\text{ and }b\in\mathbb{R}^m.
\end{gathered}$$

In [2]:
# 원래는 W와 b는 임의의 수로 초기화 되는데 여기서는 예시로 그냥 써둠
W = torch.FloatTensor([[1, 2],
                       [3, 4],
                       [5, 6]])
b = torch.FloatTensor([2, 2])

# => n = 3, m = 2이구나!

In [3]:
print(W.size())
print(b.size())

torch.Size([3, 2])
torch.Size([2])


In [4]:
def linear(x, W, b):
    y = torch.matmul(x, W) + b
    #   (N,n) x (n,m) = (N,m)    
    return y

In [5]:
x = torch.FloatTensor([[1, 1, 1],
                       [2, 2, 2],
                       [3, 3, 3],
                       [4, 4, 4]])
# => |x| = (4,3) => N = 4구나!

print(x.size())

torch.Size([4, 3])


In [6]:
y = linear(x, W, b)

In [7]:
print(y.size())

torch.Size([4, 2])


위 예시는 실제로 사용할 코드는 아니고 그냥 예시로 이런 방식으로 계산이 된다는 거를 보여주는 코드들임

## nn.Module

nn.Module이 실제로 사용할 거임!!

In [2]:
import torch.nn as nn

In [3]:
class MyLinear(nn.Module):  # nn.Module을 상속받아서 MyLinear라는 Class 생성
# "__init__"과 "forward"가 중요!

# input_dim은 n에 해당, output_dim은 m에 해당
    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__() #super()로 기반 클래스(이 경우 nn.Module)의 __init__ 메서드 호출
        
        self.W = torch.FloatTensor(input_dim, output_dim)  # |W| = (input_dim,output_dim)
        self.b = torch.FloatTensor(output_dim)  # |b| = (output_dim,)

    # You should override 'forward' method to implement detail.
    # The input arguments and outputs can be designed as you wish.
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        # 여기서 x는 Batch_size로 예시를 한 것
        
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        
        return y

In [10]:
linear = MyLinear(3, 2)  # input_dim = 3, output_dim = 2가 할당됨

y = linear(x)
# linear.forward(x)가 아니라 그냥 linear(x)만 해도 되도록 pytorch에서 구현이 되어있다!!!
# 강사님 추천: linear.forward(x) 말고 그냥 linear(x)로 하는 게 더 좋다! 자세한 거는 어려운 내용이라서 생략한다고 하심

In [11]:
print(y.size())

torch.Size([4, 2])


In [12]:
for p in linear.parameters():
    print(p)

You can see that there is no weight parameters to learn.
Above way can forward(or calculate) values, but it cannot be trained.

위에 코드도 사실은 아직 파라미터들을 Train시킬 수 있는 상태가 아님. 아래 1.2.1이 정말로 파라미터를 학습시킬 수 있는 코드임!!!

### Correct way: nn.Parameter

In [13]:
class MyLinear(nn.Module):

    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()
        
        self.W = nn.Parameter(torch.FloatTensor(input_dim, output_dim))
        self.b = nn.Parameter(torch.FloatTensor(output_dim))
        # 중요: W와 b를 할당할 때 nn.Parameter()를 써줘야 W, b를 학습시킬 수 있음!
        
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = torch.matmul(x, self.W) + self.b
        # |y| = (batch_size, input_dim) * (input_dim, output_dim)
        #     = (batch_size, output_dim)
        
        return y

Reference: https://pytorch.org/docs/stable/nn.html#torch.nn.Parameter

A kind of Tensor that is to be considered a module parameter.

Parameters are Tensor subclasses, that have a very special property when used with Module s - when they’re assigned as Module attributes they are automatically added to the list of its parameters, and will appear e.g. in parameters() iterator. Assigning a Tensor doesn’t have such effect. This is because one might want to cache some temporary state, like last hidden state of the RNN, in the model. If there was no such class as Parameter, these temporaries would get registered too.

In [14]:
linear = MyLinear(3, 2)

y = linear(x)
# linear.forward(x)가 아니라 그냥 linear(x)만 해도 되도록 pytorch에서 구현이 되어있다!!!
# 강사님 추천: linear.forward(x) 말고 그냥 linear(x)로 하는 게 더 좋다! 자세한 거는 어려운 내용이라서 생략한다고 하심

In [15]:
print(y.size())

torch.Size([4, 2])


In [16]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[ 0.0000e+00, -0.0000e+00],
        [ 9.1002e+31, -2.8586e-42],
        [ 8.4078e-45,  0.0000e+00]], requires_grad=True)
Parameter containing:
tensor([4.7428e+30, 7.1429e+31], requires_grad=True)


위 코드도 low-level로 잘 짠 코드지만 사실은 nn.Linear라고 미리 짜여진 게 있다!
실제로는 앞으로 Linear model을 만들어야 하면 아래 nn.Linear를 사용하면 된다!

## nn.Linear

In [17]:
linear = nn.Linear(3, 2)

y = linear(x)

In [18]:
print(y.size())

torch.Size([4, 2])


In [19]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[-0.3386, -0.5355, -0.4991],
        [ 0.1993,  0.4776,  0.1894]], requires_grad=True)
Parameter containing:
tensor([0.1819, 0.0941], requires_grad=True)


nn.Linear에서는 우리가 nn.module로 코드를 직접 짤 때와 다르게 우리가 했던 W가 전치된 상태 즉 (2,3) 형태다

### nn.Module can contain other nn.Module's child classes.

In [20]:
class MyLinear(nn.Module):
# nn.Module을 상속받아서 MyLinear라는 클래스를 생성
    
    def __init__(self, input_dim=3, output_dim=2):
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        super().__init__()  #super()로 기반 클래스(이 경우 nn.Module)의 __init__ 메서드 호출
        
        self.linear = nn.Linear(input_dim, output_dim)
        # nn.Module 안에서 nn.Linear라는 클래스를 또 사용함!
        # 사실 이 MyLinear라는 클래스는 진짜로 그냥 wrapping 한 번만 더 해준 거라서 별 의미는 없다.
        # 그러나, 나중에 딥하게 만들다보면 더 많은 sub 함수들을 사용하게 되니까 이렇게도 된다는 거를 보여준 예시
        
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        y = self.linear(x)
        # |y| = (batch_size, output_dim)
        
        return y

In [21]:
linear = MyLinear(3, 2)

y = linear(x)

In [22]:
print(y.size())

torch.Size([4, 2])


In [23]:
for p in linear.parameters():
    print(p)

Parameter containing:
tensor([[-0.1267,  0.0563,  0.3951],
        [ 0.2291,  0.3214,  0.2595]], requires_grad=True)
Parameter containing:
tensor([0.3659, 0.4013], requires_grad=True)
